In [27]:
import numpy
from matplotlib import pyplot as plt
import matplotlib as mpl
from scipy import signal
from multiprocessing import Pool
import json
import wfdb

In [38]:
matched_dir = "mimic3wdb-matched"
riemann_dir = "riemann"
accp = {}
with open("accp_deliberation.json", 'r') as f:
    for e in json.loads(f.read())["accp_deliberation"]:
        accp[e["hadm_id"]] = e["type"]
with open("entries.json", 'r') as f:
    entries = json.loads(f.read())
with open("signals.json", 'r') as f:
    signals = json.loads(f.read())

In [69]:
entries["BP-ECG"]["14400"][0]

[148, 34, ['II'], ['mV']]

In [52]:
def resample_multichan(xs, fs, fs_target, resamp_ann_chan=0):
    assert resamp_ann_chan < xs.shape[1]

    lx = []
    lt = None
    for chan in range(xs.shape[1]):
        xx, tt = wfdb.processing.resample_sig(xs[:, chan], fs, fs_target)
        lx.append(xx)
        if chan == resamp_ann_chan:
            lt = tt
    return numpy.column_stack(lx), lt

In [58]:
def combi(signames):
    ecgs = [s for s in signames if s in signals['ECG']]
    others = [(s, i) for i, s in enumerate(signames) if s not in ecgs]
    res = []
    for e in ecgs:
        res.append([(e, signames.index(e))] + others)
    return res

record = wfdb.rdsamp("mimic3wdb-matched/s24942/3007103_0031", physical=False)
print(record.signame)
combi(record.signame)

['RESP', 'I', 'MCL', 'II', 'PLETH', 'ABP']


[[('I', 1), ('RESP', 0), ('MCL', 2), ('PLETH', 4), ('ABP', 5)],
 [('II', 3), ('RESP', 0), ('MCL', 2), ('PLETH', 4), ('ABP', 5)]]

## Dataset Creation

In [57]:
# pre-processing
# -> Divide each record in dataset examples (one record could have many ecg channels - divide using one ecg channel by dataset example)
# -> Check signal validity
# -> Resample to same frequency
# -> Use same units (✔ all signames are of the same unit!)
# -> Compute GQRS heart-rate and RR

fs_target = 125
def f(t):
    record = wfdb.rdsamp(t, physical=False)
    new_x, lt = resample_multichan(record.d_signals, record.fs, fs_target)
    for c in combi(record.signame):
        x = new_x[:,c[0][1]]
        assert len(x) > 1000
        peak_indexes = wfdb.processing.gqrs_detect(x=x, frequency=record.fs, adcgain=record.adcgain[0], adczero=record.adczero[0], threshold=1.0)
        new_indexes = wfdb.processing.correct_peaks(x=x, peaks_indexes=peak_indexes, min_gap=record.fs*60/10, max_gap=record.fs*60/350, smooth_window=150)
        xs = numpy.column_stack([x] + [new_x[:,i] for _, i in c[1:]])
        
        if fs_target != record.fs:
            new_peaks = wfdb.processing.resample_ann(lt, new_indexes)
        else:
            new_peaks = new_indexes
        
        hrs = wfdb.processing.compute_hr(len(x), new_peaks, fs_target)
        
        print(riemann_dir + '/' + t.replace('/', '-') + '-' + c[0][0])
        numpy.save(riemann_dir + '/' + t.replace('/', '-') + '-' + c[0][0], numpy.asarray([xs, new_peaks, hrs, c]))
            

todo = []

for sigs, v in entries.items():
    for time, vv in v.items():
        for e in vv[time]:
            todo.append()
    entries[sigs]
entries["BP-ECG"]["14400"][0]
entries["BP-ECG"]["14400"][0]
Out[69]:
[148, 34, ['II'], ['mV']]
            
with Pool(5) as p:
    print(p.map(f, ["mimic3wdb-matched/s24942/3007103_0031"]))


riemann/mimic3wdb-matched-s24942-3007103_0031-I
riemann/mimic3wdb-matched-s24942-3007103_0031-II
[None]


In [6]:
seq1 = numpy.asarray([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,  0, 0, 0, 1, 0, 0])
seq2 = numpy.asarray([22, 35, 32, 29])

sequences = [seq1, seq2]

# I - Resample sequences to match the one with the higher frequency (up-sampling using Fourier method)
result = []
m = max([len(s) for s in sequences])
for s in sequences:
    result.append(signal.resample(s, m))

# II - Compute covariance matrix between discrete signals
print(numpy.cov(result[0], result[1]))

# III - 

[array([  7.94685954e-16,   1.00000000e+00,   3.98803797e-16,
        -2.80477396e-16,   4.44089210e-16,  -3.94421338e-16,
         1.64780470e-15,   1.00000000e+00,   2.33731163e-17,
         3.09693791e-16,   3.91499698e-16,  -1.86984930e-16,
         1.00000000e+00,   1.67117782e-15,  -1.66533454e-16,
         3.03850512e-16,   1.00000000e+00,   8.16598251e-16,
        -4.67462326e-16]), array([ 22.        ,  23.77216093,  26.78322187,  30.28099721,
        33.37945774,  35.36855346,  35.94900103,  35.30002795,
        33.95784054,  32.56104718,  31.57347964,  31.10215618,
        30.88568449,  30.45436107,  29.38904651,  27.56305614,
        25.25799834,  23.0979456 ,  21.82396411])]
[[  0.1754386   -0.15467379]
 [ -0.15467379  21.24305556]]
